## Скачиваем необходимое

Сначала нужно средствами NLTK загрузить WordNet.



In [1]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/muzafarov/nltk_data...


True

## Готовим данные к работе

Затем импортируем данные из подготовленного текстового файла. Файл содержит набор пар слов (только имён существительных), для которых известны экспертные оценки сходства.

Строим ассоциативный массив "пара слов -- оценка близости".

In [2]:
with open("wordsim_similarity_goldstandard.txt", encoding="utf-8") as rf:
  triples = [line.strip().split("\t") for line in rf.readlines()]
  score_map = {tuple(triple[:2]): float(triple[2]) for triple in triples}

Отметим, что из исходного набора данных мы взяли только экспертные оценки сходства (similarity) и только для существительных. Исходный набор данных доступен по [ссылке](http://alfonseca.org/pubs/ws353simrel.tar.gz)

Посмотрим на примеры оценок. 

У слов может быть по несколько значений, которые различаются в WordNet. Здесь -- ради примера -- мы будем "жадно" выбирать первое попавшееся, но далее будем работать с ними иначе.




In [16]:
for w1, w2 in list(score_map)[:2]:
  
  print("\nWords: %s-%s\nGround truth score: %.2f" % (w1, w2, score_map[(w1, w2)]))
  
  ss1 = wn.synset(w1 + ".n.01")
  ss2 = wn.synset(w2 + ".n.01")

  print("\nPath: %.3f" % ss1.path_similarity(ss2), end=" ")
  print("\nwup: %.3f" % ss1.wup_similarity(ss2), end=" ")
  print("\nshortest_path: %.3f" % ss1.shortest_path_distance(ss2))


Words: asylum-madhouse
Ground truth score: 8.87

Path: 0.125 
wup: 0.632 
shortest_path: 7.000

Words: consumer-confidence
Ground truth score: 4.13

Path: 0.071 
wup: 0.133 
shortest_path: 13.000


Вычисляем для всех пар несколько оценок

In [19]:
from itertools import product

list_pairs = list(score_map)
wup_list, true_list, path_list, lch_list = [], [], [], []

# для всех пар
for w1, w2 in list_pairs:

  try:
    all_w1 = wn.synsets(w1, pos="n")
    all_w2 = wn.synsets(w2, pos="n")

    # добавляем интересующие нас метрики и экспертные оценки
    wup = max([item1.wup_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    wup_list.append(wup)

    path = max([item1.path_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    path_list.append(path)

    lch = max([item1.lch_similarity(item2) \
                for item1, item2 in product(all_w1, all_w2)])
    lch_list.append(lch)
    
    true_list.append(score_map[(w1, w2)])

  except Exception as e:
    print(w1, w2, "error:", e)

## Вычисляем ранговую корреляцию Спирмена

In [21]:
from scipy.stats import spearmanr

coef, p = spearmanr(wup_list, true_list)
print("wup  Spearman R: %.4f" % coef)

coef, p = spearmanr(path_list, true_list)
print("path Spearman R: %.4f" % coef)

coef, p = spearmanr(lch_list, true_list)
print("lch Spearman R: %.4f" % coef)


wup  Spearman R: 0.6415
path Spearman R: 0.6278
lch Spearman R: 0.6278


In [6]:
triples

[['tiger', 'cat', '7.35'],
 ['tiger', 'tiger', '10.00'],
 ['plane', 'car', '5.77'],
 ['train', 'car', '6.31'],
 ['television', 'radio', '6.77'],
 ['media', 'radio', '7.42'],
 ['bread', 'butter', '6.19'],
 ['cucumber', 'potato', '5.92'],
 ['doctor', 'nurse', '7.00'],
 ['professor', 'doctor', '6.62'],
 ['student', 'professor', '6.81'],
 ['smart', 'stupid', '5.81'],
 ['wood', 'forest', '7.73'],
 ['money', 'cash', '9.15'],
 ['king', 'queen', '8.58'],
 ['king', 'rook', '5.92'],
 ['bishop', 'rabbi', '6.69'],
 ['fuck', 'sex', '9.44'],
 ['football', 'soccer', '9.03'],
 ['football', 'basketball', '6.81'],
 ['football', 'tennis', '6.63'],
 ['Arafat', 'Jackson', '2.50'],
 ['physics', 'chemistry', '7.35'],
 ['vodka', 'gin', '8.46'],
 ['vodka', 'brandy', '8.13'],
 ['drink', 'eat', '6.87'],
 ['car', 'automobile', '8.94'],
 ['gem', 'jewel', '8.96'],
 ['journey', 'voyage', '9.29'],
 ['boy', 'lad', '8.83'],
 ['coast', 'shore', '9.10'],
 ['asylum', 'madhouse', '8.87'],
 ['magician', 'wizard', '9.02'],
 

In [7]:
score_map

{('tiger', 'cat'): 7.35,
 ('tiger', 'tiger'): 10.0,
 ('plane', 'car'): 5.77,
 ('train', 'car'): 6.31,
 ('television', 'radio'): 6.77,
 ('media', 'radio'): 7.42,
 ('bread', 'butter'): 6.19,
 ('cucumber', 'potato'): 5.92,
 ('doctor', 'nurse'): 7.0,
 ('professor', 'doctor'): 6.62,
 ('student', 'professor'): 6.81,
 ('smart', 'stupid'): 5.81,
 ('wood', 'forest'): 7.73,
 ('money', 'cash'): 9.15,
 ('king', 'queen'): 8.58,
 ('king', 'rook'): 5.92,
 ('bishop', 'rabbi'): 6.69,
 ('fuck', 'sex'): 9.44,
 ('football', 'soccer'): 9.03,
 ('football', 'basketball'): 6.81,
 ('football', 'tennis'): 6.63,
 ('Arafat', 'Jackson'): 2.5,
 ('physics', 'chemistry'): 7.35,
 ('vodka', 'gin'): 8.46,
 ('vodka', 'brandy'): 8.13,
 ('drink', 'eat'): 6.87,
 ('car', 'automobile'): 8.94,
 ('gem', 'jewel'): 8.96,
 ('journey', 'voyage'): 9.29,
 ('boy', 'lad'): 8.83,
 ('coast', 'shore'): 9.1,
 ('asylum', 'madhouse'): 8.87,
 ('magician', 'wizard'): 9.02,
 ('midday', 'noon'): 9.29,
 ('furnace', 'stove'): 8.79,
 ('food', 'frui

In [8]:
import pandas as pd

df = pd.read_csv('Task_4_sample_2.csv')
df

,word_1,word_2,Score
0,asylum,madhouse,8.87
1,consumer,confidence,4.13
2,street,children,4.94
3,dividend,payment,7.63
4,direction,combination,2.25
...,...,...,...
145,marathon,sprint,7.47
146,development,issue,3.97
147,attempt,peace,4.25
148,sign,recess,2.38


In [13]:
score_map

{('tiger', 'cat'): 7.35,
 ('tiger', 'tiger'): 10.0,
 ('plane', 'car'): 5.77,
 ('train', 'car'): 6.31,
 ('television', 'radio'): 6.77,
 ('media', 'radio'): 7.42,
 ('bread', 'butter'): 6.19,
 ('cucumber', 'potato'): 5.92,
 ('doctor', 'nurse'): 7.0,
 ('professor', 'doctor'): 6.62,
 ('student', 'professor'): 6.81,
 ('smart', 'stupid'): 5.81,
 ('wood', 'forest'): 7.73,
 ('money', 'cash'): 9.15,
 ('king', 'queen'): 8.58,
 ('king', 'rook'): 5.92,
 ('bishop', 'rabbi'): 6.69,
 ('fuck', 'sex'): 9.44,
 ('football', 'soccer'): 9.03,
 ('football', 'basketball'): 6.81,
 ('football', 'tennis'): 6.63,
 ('Arafat', 'Jackson'): 2.5,
 ('physics', 'chemistry'): 7.35,
 ('vodka', 'gin'): 8.46,
 ('vodka', 'brandy'): 8.13,
 ('drink', 'eat'): 6.87,
 ('car', 'automobile'): 8.94,
 ('gem', 'jewel'): 8.96,
 ('journey', 'voyage'): 9.29,
 ('boy', 'lad'): 8.83,
 ('coast', 'shore'): 9.1,
 ('asylum', 'madhouse'): 8.87,
 ('magician', 'wizard'): 9.02,
 ('midday', 'noon'): 9.29,
 ('furnace', 'stove'): 8.79,
 ('food', 'frui

In [15]:
score_map = {}
for i in range(len(df)):
  score_map[(df['word_1'][i], df['word_2'][i])] = df['Score'][i]
score_map

{('asylum', 'madhouse'): 8.87,
 ('consumer', 'confidence'): 4.13,
 ('street', 'children'): 4.94,
 ('dividend', 'payment'): 7.63,
 ('direction', 'combination'): 2.25,
 ('cup', 'tableware'): 6.85,
 ('seafood', 'food'): 8.34,
 ('stock', 'egg'): 1.81,
 ('ministry', 'culture'): 4.69,
 ('journey', 'voyage'): 9.29,
 ('tiger', 'animal'): 7.0,
 ('shore', 'woodland'): 3.08,
 ('money', 'cash'): 9.15,
 ('professor', 'doctor'): 6.62,
 ('precedent', 'example'): 5.85,
 ('image', 'surface'): 4.56,
 ('noon', 'string'): 0.54,
 ('music', 'project'): 3.63,
 ('magician', 'wizard'): 9.02,
 ('museum', 'theater'): 7.19,
 ('situation', 'isolation'): 3.88,
 ('physics', 'chemistry'): 7.35,
 ('peace', 'atmosphere'): 3.69,
 ('cup', 'substance'): 1.92,
 ('Mexico', 'Brazil'): 7.44,
 ('football', 'tennis'): 6.63,
 ('computer', 'news'): 4.47,
 ('street', 'place'): 6.44,
 ('line', 'insurance'): 2.69,
 ('psychology', 'discipline'): 5.58,
 ('century', 'year'): 7.59,
 ('cell', 'phone'): 7.81,
 ('life', 'death'): 7.88,
 ('

In [22]:
from nltk.corpus import wordnet as wn

# Получаем синсет для 'doctor.n.01'
doctor_synset = wn.synset('doctor.n.01')

# Получаем гипонимы синсета 'doctor.n.01'
hyponyms = doctor_synset.hyponyms()

# Выводим количество гипонимов
print("Количество гипонимов для синсета doctor.n.01:", len(hyponyms))

# Получаем имя первого гипонима из списка
first_hyponym_name = hyponyms[0].name()

# Выводим имя первого гипонима
print("Имя первого гипонима:", first_hyponym_name)


Количество гипонимов для синсета doctor.n.01: 14
Имя первого гипонима: abortionist.n.01
